# IMG04 : Check Data avalaibility

- author Sylvie Dagoret-Campagne
- creation date 2025-10-22 :
- affiliation : IJCLab
- Kernel @usdf **w_2025_38*
- Home emac : base (conda)
- laptop : conda_py313

In [ ]:
# where are stored the figures
pathfigs = "figs_IMG04"
if not os.path.exists(pathfigs):
    os.makedirs(pathfigs) 
prefix = "img04"
figtype = ".png"

In [ ]:
FLAG_SAVEFITS = False
# where are stored the figures
pathdata = "data_IMG04"
if FLAG_SAVEFITS: 
    if not os.path.exists(pathdata):
        os.makedirs(pathdata) 
    imgtype = "fits"

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
import lsst.afw.display as afwDisplay

In [ ]:
plt.rcParams["figure.figsize"] = (16,8)
plt.rcParams["axes.labelsize"] = 'xx-large'
plt.rcParams['axes.titlesize'] = 'xx-large'
plt.rcParams['xtick.labelsize']= 'xx-large'
plt.rcParams['ytick.labelsize']= 'xx-large'
plt.rcParams["legend.fontsize"] = "xx-large"

In [ ]:
import numpy as np
import pandas as pd
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Remove to run faster the notebook
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
from IMG00_parameters import *

In [ ]:
DumpConfig()

## Butler

In [ ]:
import lsst.daf.butler as dafButler
from lsst.daf.butler import Butler

In [ ]:
butler = Butler(repo, collections=collection)

## Initialisation

## Read the Exposure File

In [ ]:
!ls ../2025-06-26-SpectractorExtraction-FromButler/data/butlerregistry	

In [ ]:
List_Of_Exposure_file = "../2025-06-26-SpectractorExtraction-FromButler/data/butlerregistry/2025-10-17_holosummary_all_filters_repo_main.csv"	

In [ ]:
df = pd.read_csv(List_Of_Exposure_file,index_col=0)

## Butler Initialisation

In [ ]:
the_collection = butlerusercollectiondict[version_run]
the_collection 

### Date selection

- choose 2025

In [ ]:
df = df[df['day_obs'] > 20250101]

### Filter selection

In [ ]:
df["filter"].unique()

In [ ]:
df2 = df[df['filter'] ==  "OG550_65mm_1~holo4_003" ]
df1 = df[df['filter'] ==  "BG40_65mm_1~holo4_003" ]

In [ ]:
df1.head()

In [ ]:
df2.head()

In [ ]:
# add time for plotting
#df_spec["Time"] = pd.to_datetime(df_spec["DATE-OBS"])

In [ ]:
#df["seq_num"]  = df["id"] % 100_000

In [ ]:
df["nightObs"] = df.apply(lambda x: x['id']//100_000, axis=1)

## night Selection

In [ ]:
#df["nightObs"].unique() 

In [ ]:
NIGHTSEL = 20250107

In [ ]:
df_sel = df[df["nightObs"] == NIGHTSEL]
df_sel = df_sel.reset_index(drop=True)

In [ ]:
df = df_sel[["id","seq_num","filter","time_start"]]
df["Time"] = pd.to_datetime(df["time_start"])

In [ ]:
df.head(n=20)

In [ ]:
EXPOSURE_SEL = 2025010700183  # BG40
#EXPOSURE_SEL = 2025010700184  #OG550

### # UPDATE THESE arguments to mirror the ones you use with command-line pipetask run
    payload:
      payloadName: 'auxtel_run_20251018_w_2025_42_spectractorv32_all_main_data_gains_holoallfilt_b'
      butlerConfig: /repo/main
      inCollection: LATISS/raw/all,refcats,LATISS/calib/legacy,LATISS/calib
      dataQuery: "instrument='LATISS' AND (exposure.day_obs>=20220301) AND physical_filter IN ('empty~holo4_003','BG40~holo4_003','FELH0600~holo4_003','SDSSr~holo4_003','BGq40_65mm_1~holo4_003','OG550_65mm_1~holo4_003') AND exposure.observation_type='science'"       
 

## Get and Plot the raw image

In [ ]:
the_raw = butler.get('raw', exposure=EXPOSURE_SEL, detector=0, collections="LATISS/raw/all", instrument='LATISS')

In [ ]:
afwDisplay.setDefaultBackend("firefly")

In [ ]:
display = afwDisplay.Display(frame=1)
# cannot succeed to show white stars on dark sky
display.setImageColormap("gray")
display.scale("asinh", "zscale")
display.mtv(the_raw.image, title="raw")    

## Get and Plot the postISRCCD from the butler

In [ ]:
the_postisrccd = butler.get('postISRCCD', exposure=EXPOSURE_SEL, detector=0, collections=the_collection , instrument='LATISS')
meta = the_postisrccd.getMetadata()
md = meta.toDict()

In [ ]:
display = afwDisplay.Display(frame=2)
# cannot succeed to show white stars on dark sky
display.setImageColormap("gray")
display.scale("asinh", "zscale")
display.mtv(the_postisrccd.image, title="postisrccd")    